In [255]:
import pandas as pd 
import psycopg2

In [256]:
def close_all_connections():
    """Function to close all active connections to the 'accounts' database."""
    conn_close = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    conn_close.set_session(autocommit=True)
    cur_close = conn_close.cursor()
    cur_close.execute("""
        SELECT pg_terminate_backend(pg_stat_activity.pid)
        FROM pg_stat_activity
        WHERE pg_stat_activity.datname = 'accounts'
          AND pid <> pg_backend_pid();
    """)
    cur_close.close()
    conn_close.close()


In [257]:
def create_database():
    # Attempt to establish a connection to the Postgres database
    try: 
        conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    except psycopg2.Error as e: 
        print("Error: Could not make connection to the Postgres database")
        print(e)
    
    # Attempt to get a cursor to the database
    try: 
        cur = conn.cursor()
    except psycopg2.Error as e: 
        print("Error: Could not get cursor to the database")
        print(e)
    
    # Set autocommit to True
    conn.set_session(autocommit=True)
    
    # Close all connections to the 'accounts' database
    close_all_connections()
    
    # Drop the existing "accounts" database if it exists
    cur.execute("DROP DATABASE IF EXISTS accounts")
    
    # Create a new "accounts" database
    cur.execute("CREATE DATABASE accounts")

    
    # Close the connection to the initial "postgres" database
    conn.close()
    
    # Connect to the newly created "accounts" database
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=root")
    
    # Create a cursor for the "accounts" database
    cur = conn.cursor()
    
    # Return the cursor and connection objects
    return cur, conn


In [258]:
def drop_tables(conn,cur):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [259]:
def create_tables(conn,cur):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [260]:
AccountsCountry=pd.read_csv(r"C:\Users\HP\Desktop\projects\project1\Wealth-AccountsCountry.csv")
AccountsCountry.head(2)

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina


In [261]:
AccountsCountry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7597 entries, 0 to 7596
Data columns (total 31 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Code                                               7592 non-null   object 
 1   Long Name                                          7594 non-null   object 
 2   Income Group                                       7593 non-null   object 
 3   Region                                             146 non-null    object 
 4   Lending category                                   107 non-null    object 
 5   Other groups                                       52 non-null     object 
 6   Currency Unit                                      146 non-null    object 
 7   Latest population census                           145 non-null    object 
 8   Latest household survey                            119 non-null    object 
 9   Special 

In [262]:
AccountsCountry_clean = AccountsCountry[["Code","Short Name","Table Name","Long Name","Currency Unit"]]
AccountsCountry_clean.head(2)

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso


In [263]:
AccountsCountry_clean["Short Name"].value_counts()

Albania        1
Romania        1
Netherlands    1
Nicaragua      1
Niger          1
              ..
Greece         1
Guatemala      1
Guinea         1
Guyana         1
Zimbabwe       1
Name: Short Name, Length: 146, dtype: int64

In [264]:
AccountData=pd.read_csv(r"C:\Users\HP\Desktop\projects\project1\Wealth-AccountData.csv")
AccountData.head(2)

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68


In [265]:
AccountData[AccountData["1995 [YR1995]"]==".."]

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
1224,Chad,TCD,"Natural capital per capita, nonrenewable asset...",NW.NCA.SAOI.PC,..,..,..,..,..,..,...,3.660951e+03,3.303945e+03,3.235367e+03,3.255143e+03,3.160460e+03,3.094919e+03,2.564788e+03,2101.72,1709.936,1716.507
1239,Chad,TCD,"Natural capital, nonrenewable assets: oil (con...",NW.NCA.SAOI.TO,..,..,..,..,..,..,...,4.230000e+10,3.950000e+10,4.000000e+10,4.160000e+10,4.180000e+10,4.230000e+10,3.620000e+10,3.06E+10,2.57E+10,2.66E+10
6369,Suriname,SUR,"Natural capital per capita, nonrenewable asset...",NW.NCA.SACO.PC,..,..,..,..,..,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0
6384,Suriname,SUR,"Natural capital, nonrenewable assets: coal (co...",NW.NCA.SACO.TO,..,..,..,..,..,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0
7409,West Bank and Gaza,PSE,"Natural capital per capita, nonrenewable asset...",NW.NCA.SACO.PC,..,..,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0
7410,West Bank and Gaza,PSE,"Natural capital per capita, nonrenewable asset...",NW.NCA.SAGA.PC,..,..,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,..
7412,West Bank and Gaza,PSE,"Natural capital per capita, nonrenewable asset...",NW.NCA.SAOI.PC,..,..,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0
7424,West Bank and Gaza,PSE,"Natural capital, nonrenewable assets: coal (co...",NW.NCA.SACO.TO,..,..,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0
7425,West Bank and Gaza,PSE,"Natural capital, nonrenewable assets: gas (con...",NW.NCA.SAGA.TO,..,..,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,..
7427,West Bank and Gaza,PSE,"Natural capital, nonrenewable assets: oil (con...",NW.NCA.SAOI.TO,..,..,0,0,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0


In [266]:
AccountData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7592 entries, 0 to 7591
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   7592 non-null   object 
 1   Country Code   7592 non-null   object 
 2   Series Name    7592 non-null   object 
 3   Series Code    7592 non-null   object 
 4   1995 [YR1995]  7592 non-null   object 
 5   1996 [YR1996]  7592 non-null   object 
 6   1997 [YR1997]  7592 non-null   object 
 7   1998 [YR1998]  7592 non-null   object 
 8   1999 [YR1999]  7592 non-null   object 
 9   2000 [YR2000]  7592 non-null   object 
 10  2001 [YR2001]  7592 non-null   object 
 11  2002 [YR2002]  7592 non-null   object 
 12  2003 [YR2003]  7592 non-null   object 
 13  2004 [YR2004]  7592 non-null   object 
 14  2005 [YR2005]  7592 non-null   float64
 15  2006 [YR2006]  7592 non-null   float64
 16  2007 [YR2007]  7592 non-null   float64
 17  2008 [YR2008]  7592 non-null   float64
 18  2009 [YR

In [267]:
AccountData_clean = AccountData[["Country Name","Country Code","Series Name","Series Code","1995 [YR1995]","2000 [YR2000]","2005 [YR2005]","2010 [YR2010]","2015 [YR2015]"]]

In [268]:
AccountData_clean.head(2)

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],2000 [YR2000],2005 [YR2005],2010 [YR2010],2015 [YR2015]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.370000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.559371e+04


In [269]:
AccountSeries=pd.read_csv(r"C:\Users\HP\Desktop\projects\project1\Wealth-AccountSeries.csv")
AccountSeries.head(2)

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


In [270]:
AccountSeries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Code                                 52 non-null     object
 1   Indicator Name                       52 non-null     object
 2   Long definition                      52 non-null     object
 3   Source                               52 non-null     object
 4   Topic                                52 non-null     object
 5   Unit of measure                      52 non-null     object
 6   Periodicity                          52 non-null     object
 7   Reference period                     52 non-null     object
 8   Statistical concept and methodology  52 non-null     object
 9   Previous Indicator Code              2 non-null      object
 10  Previous Indicator Name              2 non-null      object
dtypes: object(11)
memory usage: 4.6+ KB


In [271]:
AccountSeries_clean = AccountSeries[["Code","Topic","Indicator Name","Long definition"]]
AccountSeries_clean.head(2)

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...


In [272]:
AccountSeries_clean["Code"].value_counts()

NW.HCA.TO         1
NW.HCA.PC         1
NW.NCA.SAOI.PC    1
NW.NCA.PRAR.PC    1
NW.NCA.RNEW.PC    1
NW.NCA.AGRI.TO    1
NW.NCA.CROL.TO    1
NW.NCA.PASL.TO    1
NW.NCA.FISH.TO    1
NW.NCA.FECO.TO    1
NW.NCA.FTIM.TO    1
NW.NCA.FOSL.TO    1
NW.NCA.MANG.TO    1
NW.NCA.SSOI.TO    1
NW.NCA.SACO.TO    1
NW.NCA.SAGA.TO    1
NW.NCA.MINR.TO    1
NW.NCA.SAOI.TO    1
NW.NCA.PRAR.TO    1
NW.NCA.RNEW.TO    1
NW.NFA.TO         1
NW.NFA.PC         1
NW.PCA.TO         1
NW.PCA.PC         1
NW.TOW.TO         1
NW.NCA.MINR.PC    1
NW.NCA.SAGA.PC    1
NW.NCA.SACO.PC    1
NW.HCA.FSEM.TO    1
NW.HCA.FEMP.PC    1
NW.HCA.MEMP.PC    1
NW.HCA.FEMA.PC    1
NW.HCA.MALE.PC    1
NW.HCA.FSEM.PC    1
NW.HCA.MSEM.PC    1
NW.HCA.FEMP.TO    1
NW.HCA.MEMP.TO    1
NW.HCA.FEMA.TO    1
NW.HCA.MALE.TO    1
NW.HCA.MSEM.TO    1
NW.NCA.SSOI.PC    1
NW.NCA.TO         1
NW.NCA.PC         1
NW.NCA.AGRI.PC    1
NW.NCA.CROL.PC    1
NW.NCA.PASL.PC    1
NW.NCA.FISH.PC    1
NW.NCA.FECO.PC    1
NW.NCA.FTIM.PC    1
NW.NCA.FOSL.PC    1


In [273]:
cur, conn = create_database()

In [274]:
AccountsCountry_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7597 entries, 0 to 7596
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Code           7592 non-null   object
 1   Short Name     146 non-null    object
 2   Table Name     146 non-null    object
 3   Long Name      7594 non-null   object
 4   Currency Unit  146 non-null    object
dtypes: object(5)
memory usage: 296.9+ KB


In [275]:
songplay_create_table = ("""
    CREATE TABLE IF NOT EXISTS AccountsCountry (
        Code varchar ,
        Short_Name varchar,
        Table_Name varchar,
        Long_Name varchar,
        Currency_Unit varchar
    )
""")
cur.execute(songplay_create_table)
conn.commit()

In [276]:
AccountData_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7592 entries, 0 to 7591
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   7592 non-null   object 
 1   Country Code   7592 non-null   object 
 2   Series Name    7592 non-null   object 
 3   Series Code    7592 non-null   object 
 4   1995 [YR1995]  7592 non-null   object 
 5   2000 [YR2000]  7592 non-null   object 
 6   2005 [YR2005]  7592 non-null   float64
 7   2010 [YR2010]  7592 non-null   float64
 8   2015 [YR2015]  7592 non-null   float64
dtypes: float64(3), object(6)
memory usage: 533.9+ KB


In [277]:
AccountData_create_table = ("""
    CREATE TABLE IF NOT EXISTS AccountData (
        Country_Name varchar,
        Country_Code varchar,
        Series_Name varchar,
        Series_Code varchar,
        YR1995 varchar,
        YR2000 varchar,
        YR2005 numeric,
        YR2010 numeric, 
        YR2015 numeric
    )
""")
cur.execute(AccountData_create_table)
conn.commit()

In [278]:
AccountSeries_clean["Code"].value_counts()

NW.HCA.TO         1
NW.HCA.PC         1
NW.NCA.SAOI.PC    1
NW.NCA.PRAR.PC    1
NW.NCA.RNEW.PC    1
NW.NCA.AGRI.TO    1
NW.NCA.CROL.TO    1
NW.NCA.PASL.TO    1
NW.NCA.FISH.TO    1
NW.NCA.FECO.TO    1
NW.NCA.FTIM.TO    1
NW.NCA.FOSL.TO    1
NW.NCA.MANG.TO    1
NW.NCA.SSOI.TO    1
NW.NCA.SACO.TO    1
NW.NCA.SAGA.TO    1
NW.NCA.MINR.TO    1
NW.NCA.SAOI.TO    1
NW.NCA.PRAR.TO    1
NW.NCA.RNEW.TO    1
NW.NFA.TO         1
NW.NFA.PC         1
NW.PCA.TO         1
NW.PCA.PC         1
NW.TOW.TO         1
NW.NCA.MINR.PC    1
NW.NCA.SAGA.PC    1
NW.NCA.SACO.PC    1
NW.HCA.FSEM.TO    1
NW.HCA.FEMP.PC    1
NW.HCA.MEMP.PC    1
NW.HCA.FEMA.PC    1
NW.HCA.MALE.PC    1
NW.HCA.FSEM.PC    1
NW.HCA.MSEM.PC    1
NW.HCA.FEMP.TO    1
NW.HCA.MEMP.TO    1
NW.HCA.FEMA.TO    1
NW.HCA.MALE.TO    1
NW.HCA.MSEM.TO    1
NW.NCA.SSOI.PC    1
NW.NCA.TO         1
NW.NCA.PC         1
NW.NCA.AGRI.PC    1
NW.NCA.CROL.PC    1
NW.NCA.PASL.PC    1
NW.NCA.FISH.PC    1
NW.NCA.FECO.PC    1
NW.NCA.FTIM.PC    1
NW.NCA.FOSL.PC    1


In [279]:
AccountSeries_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Code             52 non-null     object
 1   Topic            52 non-null     object
 2   Indicator Name   52 non-null     object
 3   Long definition  52 non-null     object
dtypes: object(4)
memory usage: 1.8+ KB


In [280]:
AccountSeries_create_table = ("""
    CREATE TABLE IF NOT EXISTS AccountSeries (
        Code varchar PRIMARY KEY,
        Topic varchar,
        Indicator_Name varchar,
        Long_definition varchar
    )
""")
cur.execute(AccountSeries_create_table)
conn.commit()

In [281]:
songplay_insert_table = ("""
    INSERT INTO AccountsCountry (
        Code ,
        Short_Name ,
        Table_Name ,
        Long_Name ,
        Currency_Unit
    )
    VALUES (%s, %s, %s, %s, %s)
""")

for index, row in AccountsCountry_clean.iterrows():
    values = (row['Code'], row['Short Name'], row['Table Name'], row['Long Name'], row['Currency Unit'])
    cur.execute(songplay_insert_table, values)
    conn.commit()

In [282]:
AccountSeries_insert_table = ("""
    INSERT INTO AccountSeries (
        Code ,
        Topic ,
        Indicator_Name ,
        Long_definition
    )
    VALUES (%s, %s, %s, %s)
""")

for index, row in AccountSeries_clean.iterrows():
    values = (row['Code'], row['Topic'], row['Indicator Name'], row['Long definition'])
    cur.execute(AccountSeries_insert_table, values)
    conn.commit()

In [286]:
AccountData_insert_table = ("""
    INSERT INTO accountdata (
        Country_Name ,
        Country_Code ,
        Series_Name ,
        Series_Code ,
        YR1995 ,
        YR2000 ,
        YR2005 ,
        YR2010 , 
        YR2015 
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

for index, row in AccountData_clean.iterrows():
    values = (row['Country Name'], row['Country Code'], row['Series Name'], row['Series Code'],row['1995 [YR1995]'],row['2000 [YR2000]'],row['2005 [YR2005]'],row['2010 [YR2010]'],row['2015 [YR2015]'])
    cur.execute(AccountData_insert_table, values)
    conn.commit()